In [6]:
# import itk
# import nibabel as nib
# import numpy as np
# import nrrd

# ImagePath1 = "Z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-42-Spiromics-67503688/Markups/88-Adam/S-Final-2-label.nrrd"
# ImagePath2 = "Z:/E-Kelly-VidaSegmentals&Sublobes/3- 20Spiromics-10ForKelly-ForERS23/Case42-IA210257_H-17900_20190805_125316_20230524_121115_export/ZUNU_vida-airtree-recentered.nii.gz"
# #Image 2 is the airway while image 1 is the arterial tree. We will take image 2 and resample it using image 1 geometry
# #Image 1 is an nrrd and Image 2 is a nii.gz

# im1, header1 = nrrd.read(ImagePath1)

# image2 = nib.load(ImagePath2)
# im2 = image2.get_fdata()

# #Getting image geometries (Origin, orientation(fixed), spacing, and size)

# #Image 1 save metadata
# origin1 = header1['space origin']
# spacing1 = header1['space directions']
# size1 = im1.shape

# #Image 2 save metadata
# origin2 = image2.affine[:3, 3]
# orientation2 = image2.affine[:3, :3]
# spacing2 = image2.header.get_zooms()
# size2 = im2.shape

In [10]:
# origin1

array([-137., -298., -311.])

In [4]:
#Try reading in the arterial tree (nrrd) and airways (NIfTI) using itk instead of pynrrd and nibabel for consistency.
import itk
import nibabel as nib
import numpy as np
import nrrd

ImagePath1 = "Z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-42-Spiromics-67503688/Markups/88-Adam/S-Final-2-label.nrrd"
ImagePath2 = "Z:/E-Kelly-VidaSegmentals&Sublobes/3- 20Spiromics-10ForKelly-ForERS23/Case42-IA210257_H-17900_20190805_125316_20230524_121115_export/ZUNU_vida-airtree-recentered.nii.gz"
#Image 2 is the airway while image 1 is the arterial tree. We will take image 2 and resample it using image 1 geometry
#Image 1 is an nrrd and Image 2 is a nii.gz

# Read the NRRD image using ITK NOTE LPS IS USED INSTEAD OF RAS
im1 = itk.imread(ImagePath1, itk.SS)
origin1 = im1.GetOrigin()  # Get image origin
spacing1 = im1.GetSpacing()  # Get pixel spacing
size1 = im1.GetLargestPossibleRegion().GetSize()  # Get image size

im2 = itk.imread(ImagePath2, itk.SS)
origin2 = im2.GetOrigin()  # Get image origin
spacing2 = im2.GetSpacing()  # Get pixel spacing
direction2 = im2.GetDirection()  # Get image direction (orientation)
size2 = im2.GetLargestPossibleRegion().GetSize()  # Get image size



In [31]:
# direction2
origin2

itkPointD3 ([-174.658, -358.158, -338.1])

In [9]:
import os

# Resample im2 to match im1's origin, spacing, and size
resampler = itk.ResampleImageFilter.New(Input=im2)
resampler.SetSize([int(x) for x in size1])
resampler.SetOutputOrigin(origin1)
resampler.SetOutputSpacing(spacing1)

# Set identity transform
transform = itk.IdentityTransform[itk.D, 3].New()
resampler.SetTransform(transform)

resampler.UseReferenceImageOn()
resampler.SetReferenceImage(im1)
resampler.Update()

# Threshold the resampled image to create a binary mask
threshold_filter = itk.BinaryThresholdImageFilter.New(Input=resampler.GetOutput())
threshold_filter.SetLowerThreshold(200)  # Threshold value, adjust if needed
threshold_filter.SetUpperThreshold(256)  # Upper threshold value, adjust if needed
threshold_filter.SetInsideValue(1)
threshold_filter.SetOutsideValue(0)
threshold_filter.Update()

outputFile = "C:/Users/nichmoore/Documents/GitHub/RA4-Project/figures/airtreeResampled1.nii.gz"

if os.path.exists(outputFile):
    os.remove(outputFile)

# Save the resampled image as NIfTI
airtreeResampled = threshold_filter.GetOutput()
itk.imwrite(airtreeResampled, outputFile)